In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time
import statsmodels.api as sm
import sys

In [4]:
# !{sys.executable} -m pip install stocktrends

In [14]:
start_date = '20131231'
# end_date = '20191101'
end_date = time.strftime("%Y%m%d")

In [24]:
url ="https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=" + start_date + "&end=" + end_date
example_df = pd.read_html(url)

data = example_df[2]
df =pd.DataFrame(data)

df['Date'] =  pd.to_datetime(df['Date']) # date
df.columns = ['date','open','high','low','close','volume','market cap']
df.set_index('date',inplace=True)
df.sort_index(inplace=True)
# df['log_ret'] = np.log(df['close'] / df['close'].shift(1))
# df['return'] = df['close'].pct_change(1)
# df['vola'] = df['return'].rolling(15).std()*(360**0.5)*100

### CAGR

In [79]:
def CAGR(DF):
    df = DF.copy()
    df['daily_ret'] = df['close'].pct_change()
    df['cum_ret'] = (1 + df['daily_ret']).cumprod()
    n = len(df) / 365
    CAGR = (df['cum_ret'][-1]) ** (1/n) - 1
    return CAGR

In [40]:
CAGR(df)

0.43992024695954224

### Vola

In [59]:
def Vola(DF):
    df = DF.copy()
    df['daily_ret'] = df['close'].pct_change()
    vola = df['daily_ret'].std()
    annual_vola = vola * np.sqrt(365)
#     week_vola = vola * np.sqrt(52)
#     month_vola = vola * np.sqrt(12)
    return annual_vola

In [60]:
Vola(df)

0.7438497923156482

In [64]:
def Sharpe(DF, rf):
    df = DF.copy()
    sr = (CAGR(df) - rf) / Vola(df)
    return sr

In [68]:
Sharpe(df, 0)

0.43992024695954224

In [86]:
def Sortino(DF, rf):
    df = DF.copy()
    df['daily_ret'] = df['close'].pct_change()
    neg_vola = df[df['daily_ret'] < 0]['daily_ret'].std() * np.sqrt(365)
    sortino = (CAGR(df) - rf) / neg_vola
    return sortino

In [87]:
Sortino(df, 0)

0.7710621205466828

### Max Drawdown

In [118]:
def max_dd(DF):
    df = DF.copy()
    df['daily_ret'] = df['close'].pct_change()
    df['cum_ret'] = (1 + df['daily_ret']).cumprod()
    df['cum_roll_max'] = df['cum_ret'].cummax()
    df['drawdown'] = df['cum_roll_max'] - df['cum_ret']
    df['drawdown_pct'] = df['drawdown'] / df['cum_roll_max']
    max_dd = df['drawdown_pct'].max()
    return max_dd

In [119]:
max_dd(df)

0.833990173048714

In [126]:
def Calmar(DF):
    df = DF.copy()
    clmr = CAGR(df) / max_dd(df)
    return clmr    

In [127]:
Calmar(df)

0.5274885258556232